# What are the key trends and factors influencing the enrolment in undergraduate programs at LSE? 


## Data Acquisition

In [128]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def get_course_numbers(url, year):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    content = soup.find("div", attrs={"class": "right-container"})
    links = content.find_all("a", href=True)
    course_list = []
    for link in links:
        text = link.text.strip()
        if len(text) > 2:
            course_list.append(text[0:6].strip())

    service = Service("chromedriver.exe")
    driver = webdriver.Chrome(service=service)

    course_enrolment = {}

    for course in course_list:
        course_url = f"https://www.lse.ac.uk/resources/calendar{year}-{year+1}/courseGuides/{course[0:2]}/{year}_{course}.htm"
        driver.get(course_url)
        try:
            total_students = WebDriverWait(driver, 0.4).until(
                EC.presence_of_element_located((By.XPATH, "//div[@id='keyFacts-Content']/p[2]"))
            ).text
            course_enrolment[course] = total_students.split(":")[-1].strip()
            
        except:
            print(f"cant find {course} in {year}")

    driver.quit()
    df = pd.DataFrame.from_dict(course_enrolment, orient="index")
    return df
course_numbers_2024_df = get_course_numbers("https://www.lse.ac.uk/resources/calendar2024-2025/courseGuides/undergraduate.htm", 2024)
course_numbers_2023_df = get_course_numbers("https://www.lse.ac.uk/resources/calendar2024-2025/courseGuides/undergraduate.htm", 2023)
course_numbers_2022_df = get_course_numbers("https://www.lse.ac.uk/resources/calendar2024-2025/courseGuides/undergraduate.htm", 2022)
course_numbers_2021_df = get_course_numbers("https://www.lse.ac.uk/resources/calendar2024-2025/courseGuides/undergraduate.htm", 2021)
course_numbers_2020_df = get_course_numbers("https://www.lse.ac.uk/resources/calendar2024-2025/courseGuides/undergraduate.htm", 2020)
course_numbers_2019_df = get_course_numbers("https://www.lse.ac.uk/resources/calendar2024-2025/courseGuides/undergraduate.htm", 2019)
course_numbers_2018_df = get_course_numbers("https://www.lse.ac.uk/resources/calendar2024-2025/courseGuides/undergraduate.htm", 2018)
course_numbers_2017_df = get_course_numbers("https://www.lse.ac.uk/resources/calendar2024-2025/courseGuides/undergraduate.htm", 2017)
course_numbers_2016_df = get_course_numbers("https://www.lse.ac.uk/resources/calendar2024-2025/courseGuides/undergraduate.htm", 2016)
print("doneeeeeeeeeeeeeeeeeeeeeeeee")

cant find LSE100
cant find LSE100
cant find LSE100
cant find AC205
cant find AC206
cant find AN253
cant find AN273
cant find AN285
cant find AN286
cant find AN287
cant find AN288
cant find AN379
cant find AN389
cant find AN390
cant find AN395
cant find AN3A1
cant find DS205
cant find EC328
cant find FM210
cant find FM211
cant find FM214
cant find FM215
cant find FM310
cant find FM311
cant find GV333
cant find GV362
cant find GV3L6
cant find HY248
cant find HY249
cant find HY344
cant find IR374
cant find IR392
cant find LL150
cant find LL200
cant find LL213
cant find LL216
cant find LL217
cant find LL220
cant find LL224
cant find LL225
cant find LL228
cant find LL229
cant find LL243
cant find LL244
cant find LL245
cant find LL279
cant find LL280
cant find LL304
cant find LL306
cant find LL307
cant find LL332
cant find LSE100
cant find LSE100
cant find LSE100
cant find MA221
cant find PH241
cant find SO243
cant find ST101
cant find ST111
cant find ST360
cant find AC105
cant find AC106
ca

for data cleaning, there are a lot of courses which have missing years. so maybe use only the courses where 4 or more recorded students numbers are present and use it like that

In [137]:
print(course_numbers_2021_df)
print(course_numbers_2023_df)

                 0
AC102          579
AC103          369
AC311          120
AC312          132
AC331          179
...            ...
ST311  Unavailable
ST312  Unavailable
ST326           60
ST327           62
ST330           74

[439 rows x 1 columns]
                 0
AC102          591
AC103          224
AC105  Unavailable
AC106  Unavailable
AC311          146
...            ...
ST313           13
ST314  Unavailable
ST326           53
ST327           61
ST330           67

[511 rows x 1 columns]
